In [ ]:
# Install necessary libraries
!pip install -q transformers accelerate evaluate bert_score rouge_score sacremoses nltk psutil pandas sacrebleu sentencepiece unbabel-comet

from huggingface_hub import login
login(token='hugging face token goes here')  # Replace with your Hugging Face token

# Download NLTK data
import nltk
nltk.download('punkt')


# Imports
import time
import torch
import psutil
import logging
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate

logging.getLogger("transformers").setLevel(logging.ERROR)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paragraph data
paragraph_data = [
    {"lang": "ar", "name": "Arabic", "src": "أنا أتقدم للحصول على تأشيرة سياحية لزيارة الولايات المتحدة. أنوي البقاء لمدة أسبوعين وزيارة معالم مشهورة مثل تمثال الحرية وساحة التايمز. سأعود إلى بلدي بعد انتهاء عطلتي.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "es", "name": "Spanish", "src": "Estoy solicitando una visa de turista para visitar los Estados Unidos. Tengo la intención de quedarme dos semanas y visitar lugares famosos como la Estatua de la Libertad y Times Square. Regresaré a mi país después de mis vacaciones.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "fr", "name": "French", "src": "Je demande un visa de touriste pour visiter les États-Unis. J’ai l’intention de rester deux semaines et de visiter des sites célèbres comme la Statue de la Liberté et Times Square. Je retournerai dans mon pays après mes vacances.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "ru", "name": "Russian", "src": "Я подаю заявку на туристическую визу для поездки в Соединенные Штаты. Я собираюсь остаться на две недели и посетить известные достопримечательности, такие как Статуя Свободы и Таймс-сквер. Я вернусь в свою страну после отпуска.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "ro", "name": "Romanian", "src": "Solicit o viză turistică pentru a vizita Statele Unite. Intenționez să stau două săptămâni și să vizitez repere populare precum Statuia Libertății și Times Square. Mă voi întoarce în țara mea după vacanță.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "bg", "name": "Bulgarian", "src": "Кандидатствам за туристическа виза за посещение на Съединените щати. Възнамерявам да остана две седмици и да посетя популярни забележителности като Статуята на свободата и Таймс Скуеър. Ще се върна в родината си след ваканцията.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "cs", "name": "Czech", "src": "Žádám o turistické vízum pro návštěvu Spojených států. Mám v úmyslu zůstat dva týdny a navštívit známá místa jako Sochu svobody a Times Square. Po dovolené se vrátím do své domovské země.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "da", "name": "Danish", "src": "Jeg ansøger om et turistvisum for at besøge USA. Jeg har til hensigt at blive i to uger og besøge kendte vartegn som Frihedsgudinden og Times Square. Jeg vender tilbage til mit hjemland efter ferien.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "de", "name": "German", "src": "Ich beantrage ein Touristenvisum, um die Vereinigten Staaten zu besuchen. Ich beabsichtige, zwei Wochen zu bleiben und bekannte Sehenswürdigkeiten wie die Freiheitsstatue und den Times Square zu besuchen. Nach meinem Urlaub kehre ich in mein Heimatland zurück.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "et", "name": "Estonian", "src": "Taotlen turismiviisat Ameerika Ühendriikide külastamiseks. Kavatsen jääda kaheks nädalaks ja külastada kuulsaid vaatamisväärsusi nagu Vabadussammas ja Times Square. Pärast puhkust naasen oma koduriiki.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
]

# Lang code mapping for NLLB
LANG_CODE_TO_ID = {
    "ar": "arb_Arab",
    "es": "spa_Latn",
    "fr": "fra_Latn",
    "ru": "rus_Cyrl",
    "ro": "ron_Latn",
    "bg": "bul_Cyrl",
    "cs": "ces_Latn",
    "da": "dan_Latn",
    "de": "deu_Latn",
    "et": "est_Latn",
    "en": "eng_Latn",
}

# Load model and tokenizer
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
model.eval()

# Load evaluation modules
bleu_mod = evaluate.load("bleu")
bertscore_mod = evaluate.load("bertscore")
meteor_mod = evaluate.load("meteor")
rouge_mod = evaluate.load("rouge")
ter_mod = evaluate.load("ter")
comet_mod = evaluate.load("comet")

# System monitor
def measure_system():
    proc = psutil.Process()
    return proc.memory_info().rss / 1024**2, psutil.cpu_percent(interval=None)

# Translation function
def translate_nllb(text: str, src_lang: str, tgt_lang: str = "en"):
    tokenizer.src_lang = LANG_CODE_TO_ID[src_lang]
    encoded = tokenizer(text, return_tensors="pt").to(device)
    start = time.perf_counter()
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(LANG_CODE_TO_ID[tgt_lang])
    )
    mid = time.perf_counter()
    pred = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    end = time.perf_counter()
    return pred, mid - start, end - start, len(tokenizer.tokenize(text))

# Evaluation loop
for item in paragraph_data:
    src_lang, name, src, ref = item["lang"], item["name"], item["src"], item["ref"]
    print(f"\n=== {name} → English ===")

    ram_before, cpu_before = measure_system()
    pred, t_first, t_total, token_count = translate_nllb(src, src_lang)
    ram_after, cpu_after = measure_system()

    # Accuracy metrics
    bleu_score = bleu_mod.compute(predictions=[pred], references=[[ref]])["bleu"]
    bert_res = bertscore_mod.compute(predictions=[pred], references=[ref], lang="en")
    meteor_score = meteor_mod.compute(predictions=[pred], references=[[ref]])["meteor"]
    rouge_score = rouge_mod.compute(predictions=[pred], references=[ref])["rougeL"]
    ter_score = ter_mod.compute(predictions=[pred], references=[[ref]])["score"]
    comet_res = comet_mod.compute(sources=[src], predictions=[pred], references=[ref])
    comet_score = next(v for v in comet_res.values() if isinstance(v, (int, float)))

    # Output
    print(f"Source    : {src}")
    print(f"Reference : {ref}")
    print(f"Predicted : {pred}\n")
    print(f"BLEU      : {bleu_score:.4f}")
    print(f"BERTScore : {bert_res['f1'][0]:.4f}")
    print(f"METEOR    : {meteor_score:.4f}")
    print(f"ROUGE-L   : {rouge_score:.4f}")
    print(f"TER       : {ter_score:.4f}")
    print(f"COMET     : {comet_score:.4f}\n")
    print(f"Time to 1st tok : {t_first:.4f}s")
    print(f"Total inf time  : {t_total:.4f}s")
    print(f"Tokens/sec      : {token_count / t_total:.2f}")
    if device.type == "cuda":
        print(f"VRAM used       : {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
    print(f"RAM   : {ram_after - ram_before:.2f} MB")
    print(f"CPU   : {cpu_after - cpu_before:.2f} %")
    print(f"Model size : {sum(p.numel() for p in model.parameters()) * 4 / 1024 ** 2:.2f} MB")
    print("=" * 60)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Using device: cpu


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`


...

Source    : Jeg ansøger om et turistvisum for at besøge USA. Jeg har til hensigt at blive i to uger og besøge kendte vartegn som Frihedsgudinden og Times Square. Jeg vender tilbage til mit hjemland efter ferien.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I'm applying for a tourist visa to visit the United States, and I plan to stay for two weeks and visit famous landmarks like the Statue of Liberty and Times Square, and I'll be returning to my home country after my vacation.

BLEU      : 0.6402
BERTScore : 0.9848
METEOR    : 0.8474
ROUGE-L   : 0.8471
TER       : 26.8293
COMET     : 0.9046

Time to 1st tok : 20.3614s
Total inf time  : 20.3618s
Tokens/sec      : 2.50
RAM   : 32.64 MB
CPU   : -4.40 %
Model size : 2346.32 MB

=== German → English ===


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Source    : Ich beantrage ein Touristenvisum, um die Vereinigten Staaten zu besuchen. Ich beabsichtige, zwei Wochen zu bleiben und bekannte Sehenswürdigkeiten wie die Freiheitsstatue und den Times Square zu besuchen. Nach meinem Urlaub kehre ich in mein Heimatland zurück.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I'm applying for a tourist visa to visit the United States. I plan to stay for two weeks and visit famous landmarks like the Statue of Liberty and Times Square. After my vacation, I'll return to my home country.

BLEU      : 0.6703
BERTScore : 0.9914
METEOR    : 0.8948
ROUGE-L   : 0.8293
TER       : 21.9512
COMET     : 0.9227

Time to 1st tok : 19.5806s
Total inf time  : 19.5810s
Tokens/sec      : 3.32
RAM   : 32.53 MB
CPU   : -1.80 %
Model size : 2346.32 MB

=== Estonian → English

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Source    : Taotlen turismiviisat Ameerika Ühendriikide külastamiseks. Kavatsen jääda kaheks nädalaks ja külastada kuulsaid vaatamisväärsusi nagu Vabadussammas ja Times Square. Pärast puhkust naasen oma koduriiki.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I'm applying for a tourist visa to visit the United States, and I plan to stay for two weeks and visit famous sights like Liberty Square and Times Square, and after my vacation, I'll be back in my home country.

BLEU      : 0.4295
BERTScore : 0.9748
METEOR    : 0.6064
ROUGE-L   : 0.6747
TER       : 53.6585
COMET     : 0.8924

Time to 1st tok : 20.7165s
Total inf time  : 20.7169s
Tokens/sec      : 2.56
RAM   : 31.21 MB
CPU   : 9.70 %
Model size : 2346.32 MB
